# Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

This notebook can run on a low-cost or free T4 runtime.


In [1]:
!pip install -q --upgrade bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.2 MB/s eta 0:00:00


`bitsandbytes` is Hugging Face library that deals with **Quantization**

In [2]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [3]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [13]:
# New capability - connect this Colab to your Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3
# Place the file on your drive in a folder called llms, and call it denver_extract.mp3

drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/test_dumps/denver_extract.mp3"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# instruct models and 1 reasoning model

# Llama 3.1 is larger and you should already be approved
# see here: https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct

# LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Llama 3.2 is smaller but you might need to request access again
# see here: https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct

LLAMA = "meta-llama/Llama-3.2-1B-Instruct"

PHI = "microsoft/Phi-4-mini-instruct"
GEMMA = "google/gemma-3-270m-it"
QWEN = "Qwen/Qwen3-4B-Instruct-2507"
DEEPSEEK = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# STEP 1: Transcribe Audio

## Option 1: Use Open Source for Transcription - Hugging Face Pipelines

In [14]:
from transformers import pipeline

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-medium.en",
    dtype=torch.float16,
    device='cuda',
    return_timestamps=True
)

result = pipe(audio_filename)
transcription = result["text"]
print(transcription)

Device set to use cuda
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.


 kind of the confluence of this whole idea of a Confluence Week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So I'll let you see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So, thank you. Thank you so much and thanks for your leadership. All right, welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the 

In [15]:
transcription

" kind of the confluence of this whole idea of a Confluence Week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So I'll let you see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So, thank you. Thank you so much and thanks for your leadership. All right, welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the

In [16]:
open_source_transcription = transcription

# STEP 2: Analyze & Report

In [17]:
system_message = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""

user_prompt = f"""
Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [18]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [20]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 Nov 2025

You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
 kind of the confluence of this whole idea of a Confluence Week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So I'll let you see the creation of the logo here. Yeah, so that basically kind of sums up

In [21]:
response = tokenizer.decode(outputs[0])

In [22]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 Nov 2025

You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
 kind of the confluence of this whole idea of a Confluence Week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So I'll let you see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So, thank you. Thank you so much and thanks for your leadership. All right, welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America, and to the republic for which it stands, one nation under God, indivisible, with liberty and justice for all. All right, thank you, Councilman Lopez, Madam Secretary roll call. Black. Clark. Here. Espinosa. Here. Here Flynn Gilmore here Here cashman here can each Lopez yeah new Ortega here Sussman mr. President here 11 present 11 members present we do have a quorum approval the minutes are there any corrections to the minutes of October 2nd Seeing none minutes of October 2nd stand approved council announcements. Are there any announcements by members of council? Councilman Clark, thank you. Mr. President. I just wanted to invite everyone down to the first ever Halloween parade on Broadway and lucky district 7 it will happen on Saturday, October 21st at 6 o'clock p.m It will move along Broadway from 3rd to Alameda It's gonna be a fun family-friendly event. Everyone's invited to come down wear a costume there will be candy for the kids and there are tiki zombies and 29 hearses and all kinds of fun and funky stuff on The fun and funky part of Broadway. So please join us October 21st at 6 o'clock for the Broadway Halloween bread. Thank you Mr. President. All right. Thank you councilman Clark. I will be there. All right Presentations, Madam Secretary, do we have any presentations? None, Mr. President. Communications, do we have any communications? None, Mr. President. We do have one proclamation this evening, Proclamation 1127, an observance of the annual Indigenous Peoples Day in the City and County of Denver. Councilman Lopez, will you please read it? Thank you, Mr. President, with pride. proclamation number 17, well let me just say this differently, proclamation number 1127 series of 2017 in observance of the second annual Indigenous Peoples Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the indigenous peoples have lived and flourished on the lands known as the America since time immemorial and that Denver and the surrounding communities are built upon the ancestral homelands of numerous indigenous tribes, which include the Southern Ute, the Ute Mountain Ute tribes of Colorado, and whereas the tribal homelands and seasonal encampments of the Arapaho and Cheyenne people along the banks of the Cherry Creek and South Platte River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes and the City and County of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas, on October 3rd, 2016, the City and County of Denver unanimously passed Council Bill 801, Series of 2016, officially designating the second Monday of October of each year as Indigenous Peoples Day in Denver, Colorado. And whereas, the Council of the City and County of Denver continues to recognize and value the vast contributions made to community made to the community through indigenous people's knowledge, science, philosophy, arts and culture and through these contributions the City of Denver has developed and thrived. Whereas the indigenous community especially you have made great efforts this year to draw attention to the contributions of indigenous people including Confluence once a week, drawing record attendance to a national Indigenous youth leadership conference, leading conversations on inclusion with their peers, and supporting increased Indigenous youth participation in science and engineering. Now therefore, be it proclaimed by the Council of the City and County of Denver, Section 1, that the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of Indigenous people to our history, our past, our present, future and continues to promote the education of the Denver community about these historical and contemporary contributions of indigenous people. Section 2, that the City and County of Denver, Colorado does hereby observe October 9, 2017 as Indigenous Peoples' Day. Section 3, that the Clerk of the City and County of Denver shall attest and affix the seal of the city and county of Denver to this proclamation and that a copy be transmitted to the Denver American Indian Commission, the City and County of Denver School District number one, and the Colorado Commission on Indian Affairs. Thank you, Councilman Lopez. Your motion to adopt? Mr. President, I move that proclamation number 1127 series of 2017 be adopted. All right, it has been moved and seconded. Council members of Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples Day in Denver officially for the second time. It's always awesome to be able to see not just this proclamation come by my desk, but to see so many different people from our community in our council chambers. It was a very beautiful piece of artwork that you presented to us earlier. And it is exactly the spirit that we drafted this proclamation. And the ordinance that created Indigenous Peoples' Day, when we sat down and wrote it as a community, we couldn't think of anything else to begin except for the confluence of the two rivers. And those confluence of the two rivers created such a great city. And we live in such an amazing city, and we're all proud of it. And sometimes we, and a lot of people from all over the country are all over the world are proud of it. And sometimes a little too proud of it to tell them to go back home. But I'm kidding when I say that. But the really nice thing about this is that we are celebrating Indigenous People's Day out of pride for who we are, who we are as a city and the contributions of indigenous people to the city, not out of spite, not out of a replacement of one culture over the other, or out of contempt or disrespect. I think of a quote that Cesar Chavez made very popular. And it stuck with me for a very long time. And any time I have the opportunity, I just speak in front of children and especially children in our community that, you know, they often second guess themselves and where they're coming from and who they are. And I always say that, you know, it's very important to be proud of who you're from. And the quote that I use from Cesar Chavez is, you know, pride in one's own culture does not require contempt or disrespect of another, right? And that's very important. It's very important for us to recognize that no matter who we are, where we come from in this society, that your pride in your own culture doesn't require, should not require the contempt or disrespect of another. And man, what a year for that to just sit on our shoulders for a while, for us to think about, right? And so I wanted to just to thank you all, I think the commission, there's going to be a couple of individuals that are going to come and speak thank you for your art, your lovely artwork for us to see what's in your heart and what now has become probably is going to be a very important symbol for the community. And also just for the work, the daily work, every single day. We still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners, right, in poverty without access to services, right? Without access to sobriety or even housing or jobs. And what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores, right? So we have a lot of work to do. And these kind of proclamations in this day is not a day off, it's a day on in Denver, right? and addressing those critical issues. So I know that my colleagues are very supportive. I'm going to ask you to support this proclamation. I know you always have done in the past. I'm very proud of today. Oh, and we made Time Magazine and Newsweek once again today as being a leader in terms of the cities that are celebrating Indigenous Peoples Day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Councilman Martella. Mr. President, I want to ask that my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was created and had the benefit of being able to go down to the four corners for a peace treaty signing ceremony between the Utes and the Comanches that had been sort of at odds with each other for about 100 years and just being able to participate in that powwow was pretty awesome. So, and for those of you who continue to participate in the annual powwow, it's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on as well as the native language from generation to generation is just so incredible because in so many cultures, people have come here and assimilated to the norms here you know the norms here and they lose their language and and and lose a lot of the culture and in the native community that that hasn't happened that has that you know commitment to just passing that on from generation to generation is is so important and so I'm happy to be a co-sponsor of this tonight thank you All right. Thank you, Councilwoman Ortega. Councilwoman Knitsche. Thank you very much, and I also want to thank my colleague for bringing this forward. And I just wanted to say a word to the artist about how beautiful and moving I thought this logo was and your description of it. And I think one of the things that is clear is, you know, the words sometimes don't convey the power of imagery or music or the other pieces that make up culture, and so I think the art is so important. And when you talked about water I was also thinking about land and I guess I just wanted to say thank you many of the Native American peoples of Colorado have been at the forefront or actually nationally of defending some of the the public lands that have been protected over the last few years that are under attack right now and they're places that you are the communities have fought to protect but that everyone gets to enjoy and so I just think that it's an example of where cultural preservation intersects with environmental protection with you know recreation and all of the other ways that that public lands are so important and so I think I just wanted to say thank you for that because I think we have some very sacred places in our country that are at risk right now and so as we celebrate I appreciate that there's still a piece of resistance in here and I think that I just want to mention a solidarity and I mentioned a feeling of solidarity with that resistance. So, thank you and happy Confluence Week. Thank you, Councilwoman Kniech. And seeing no other comments, I'll just say a couple. And in a time of such divisive ugliness and just despicable behavior from our leadership, the reason I'm so supportive of Indigenous Peoples' Day is because it means inclusivity. It means respecting all, respecting those who have been silenced on purpose for a long time and whose history has not been told. And so we celebrate inclusivity in the face of such evil times, honestly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Summary**
- The Denver City Council meeting was attended by various council members, including Councilman Lopez, who presented a proclamation to celebrate Indigenous Peoples Day.
- The proclamation recognizes the cultural and historical contributions of Indigenous peoples to Denver and the surrounding areas.
- The council members also discussed the importance of addressing critical issues such as poverty, lack of access to services, and environmental protection.

**Discussion Points**
- The council members discussed the importance of recognizing and valuing the contributions of Indigenous peoples to the city and state.
- They also touched on the theme of confluence, which was introduced by Councilman Lopez, and how it relates to the creation of the logo.
- The council members expressed their pride and gratitude for the contributions of Indigenous peoples to Denver and the surrounding areas.
- They also acknowledged the challenges of addressing poverty, lack of access to services, and environmental protection in the city.

**Takeaways**
- The council members acknowledged the importance of recognizing and valuing the contributions of Indigenous peoples to Denver and the surrounding areas.
- They also emphasized the need to address critical issues such as poverty, lack of access to services, and environmental protection.
- The council members expressed their gratitude for the contributions of Indigenous peoples to Denver and the surrounding areas.

**Action Items**
- Councilman Lopez proposed that the council adopt the proclamation and make it official.
- Councilwoman Ortega supported the proposal and expressed her gratitude for the contributions of Indigenous peoples to Denver.
- The council members also discussed the importance of addressing poverty, lack of access to services, and environmental protection in the city.
- They also acknowledged the challenges of addressing these issues and expressed their commitment to making a positive change.<|eot_id|>